In [2]:
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

from textwrap import dedent
import json, re

In [1]:
from datasets import load_dataset

dataset = load_dataset("diwank/airoboros-agent")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--airoboros-agent-f0fe81129258f7ee/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
all_fns = [
    json.loads(row["chatml"][1]["content"])
    for row in dataset["train"]
]

In [20]:
kk = [
    list(f.keys())
    for f in all_fns 
    if isinstance(f, dict)
]

all_fn_names_set = set([n for fl in kk for n in fl])
all_fn_names = [name for name in all_fn_names_set if " " not in name]

In [21]:
all_fn_names

['weather_forecaster',
 'data_filter',
 'get_articles',
 'email_notification',
 'weather_data_analysis',
 'resize_image',
 'file_converter',
 'search_articles',
 'data_cleaning',
 'blog_post_analysis',
 'email_notifications',
 'csv_parser',
 'calculator',
 'document_analysis',
 'word_count',
 'image_search',
 'image_manipulation',
 'topic_modeling',
 'multi_item_purchase_analysis',
 'sales_report_analysis',
 'blog_filter',
 'article_search',
 'age_filter',
 'image_classification',
 'trending_analysis',
 'server_monitoring',
 'employee_lookup',
 'image_recognition',
 'web_search',
 'term_count',
 'email_response',
 'text_parser',
 'stock_data',
 'employee_distribution',
 'email_filter',
 'video_filter',
 'json_parser',
 'inventory_management',
 'sentiment_analysis',
 'email_analytics',
 'blog_search',
 'get_posts',
 'data_extraction',
 'color_adjuster',
 'file_reader',
 'file_upload',
 'movie_recommendation',
 'sort',
 'document_filter',
 'data_visualization',
 'csv_analytics',
 'weathe

In [4]:
def make_jsc_prompts(row):
    chatml = row["chatml"]

    examples = [
        json.loads(msg["content"])
        for msg in chatml[2:]
        if msg["content"].startswith("{")
    ]
    
    functions = json.loads(chatml[1]["content"])

    if isinstance(functions, dict):
        functions = [{"name": name, **defn} for name, defn in functions.items()]
    else:
        functions = [
            {"name": name, **defn}
            for fn in functions
            for name, defn in fn.items()
            if isinstance(defn, dict)
        ]

    pairs = []
    for fn in functions:
        matching_example = next(
            (e for e in examples if e["function"] == fn["name"]),
            None,
        )

        pairs.append((fn, matching_example))

    prompts = []
    for fn, example in pairs:
        name = fn["name"]
        yaml_defn = yaml.dump(fn, Dumper=Dumper)
        yaml_example = yaml.dump(example, Dumper=Dumper) if example else None
        
        prompt = f"""\
We have a function `{name}` whose signature is described by the yaml given below.

Function definition:
```yaml
{yaml_defn}
```

"""

        prompt += f"""\
Example usage:
```yaml
{yaml_example}
```

""" if yaml_example else ""

        prompt += f"""\
Convert the yaml description for `name` into its standard, valid jsonschema representation. Surround your answer in triple backticks (```).
"""
        
        prompts.append(prompt.strip())

    return dict(jsc_prompts=prompts)

dataset = dataset.map(make_jsc_prompts)

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--airoboros-agent-6c545f4eeccae859/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-6b8a222e709e9480.arrow


In [5]:
def make_jsc_schema(row):
    
    from textwrap import dedent
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_random_exponential,
    )

    import openai
    openai.api_key = "XXX"

    from redis import StrictRedis
    from redis_cache import RedisCache

    client = StrictRedis(host="localhost", decode_responses=True)
    cache = RedisCache(redis_client=client)

    chatml = row["chatml"]
    jsc_prompts = row["jsc_prompts"]
    
    @cache.cache()
    @retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(12))
    def completion_with_backoff(prompt):
        messages = [
            dict(
                role="system",
                content=prompt,
            ),
        ]
        
        completion = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
        )
        
        result = completion.choices[0].message["content"]
        return result

    jsc_results = [
        completion_with_backoff(prompt)
        for prompt in jsc_prompts
    ]
    
    row["jsc_results"] = jsc_results
    return row

dataset = dataset.map(make_jsc_schema, num_proc=20)

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--airoboros-agent-6c545f4eeccae859/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c017f4f9250abeb3_*_of_00020.arrow


In [6]:
dataset["train"][0]["jsc_results"]

['```json\n{\n  "type": "object",\n  "properties": {\n    "name": {\n      "type": "string",\n      "description": "Helps locate information by transforming the input query into relevant search terms and filters."\n    },\n    "parameters": {\n      "type": "object",\n      "properties": {\n        "alternatives": {\n          "type": "array",\n          "items": {\n            "type": "string"\n          },\n          "description": "A list of alternate important keywords or phrases from the input, like acronyms, common alternate synonyms, etc."\n        },\n        "search_terms": {\n          "type": "array",\n          "items": {\n            "type": "string"\n          },\n          "description": "A list of crucial keywords or phrases from the input."\n        }\n      }\n    },\n    "date_range": {\n      "type": "object",\n      "properties": {\n        "begin": {\n          "type": "string",\n          "description": "Restricts results to items dated on or after this value, if

In [7]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

newline = '\n'
available_functions = lambda fns: dict(
    role="system",
    name="functions",
    content=f"""\
Available functions:
{(newline*2).join([json.dumps(fn, indent=2) for fn in fns])}"""
)

function_call = lambda name, arguments: dict(
    role="function_call",
    content=json.dumps(dict(name=name, arguments=arguments)),
)

In [8]:
json_verify = lambda x, indent=2: json.dumps(
    json.loads(x),
    indent=indent,
)

def replace_with_jsc(row):
    jsc_results = row["jsc_results"]
    old_chatml = row["chatml"]

    results = [
        re.split(r"```\w*(?:$|\n)", jsc)[1]
        for jsc in jsc_results
    ]

    schemas = [json_verify(result) for result in results]
    schemas = "\n\n".join(schemas)

    chatml = [
        situation(
            "A user is talking to their helpful AI Assistant that can help them accomplish different tasks."
            " In order to do so, the AI has access to various functions that it can call as described below."
            " The AI reasons about the tasks and chooses the appropriate function and its input in order to gather information or take actions."
            "\n\nIn order to call a function, AI just needs to specify the name of the function to call and its arguments as a valid JSON string."
        ),
        information(f"Available functions:\n\n{schemas}"),
    ]

    for msg in old_chatml[2:]:
        content = msg["content"]
        if content.startswith('{'):
            chatml.append(dict(
                role="function_call",
                content=content
            ))

        else:
            chatml.append(msg)

    return dict(chatml=chatml)

In [9]:
dataset = dataset.map(replace_with_jsc).remove_columns(set(dataset["train"].column_names) - {"chatml"})

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--airoboros-agent-6c545f4eeccae859/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-18e10ac11d3928ae.arrow


In [10]:
from pprint import pprint
pprint(dataset["train"][1]["chatml"])

[{'content': 'A user is talking to their helpful AI Assistant that can help '
             'them accomplish different tasks. In order to do so, the AI has '
             'access to various functions that it can call as described below. '
             'The AI reasons about the tasks and chooses the appropriate '
             'function and its input in order to gather information or take '
             'actions.\n'
             '\n'
             'In order to call a function, AI just needs to specify the name '
             'of the function to call and its arguments as a valid JSON '
             'string.',
  'name': 'situation',
  'role': 'system'},
 {'content': 'Available functions:\n'
             '\n'
             '{\n'
             '  "description": "This tool is useful in performing various '
             'aggregations, counts, etc. from CSV data.",\n'
             '  "type": "object",\n'
             '  "properties": {\n'
             '    "name": {\n'
             '      "type": "

In [22]:
def fix_functions(row):
    chatml = row["chatml"]
    funcs_content = chatml[1]["content"]
    funcs_content = funcs_content.replace("Available functions:", "").strip()

    funcs = [
        json.loads(s) for s in funcs_content.split('\n\n')
    ]

    fixed_funcs = []
    for func in funcs:
        if func.get("type") == "string":
            continue

        if func.get("name"):
            func["parameters"] = func["params"]
            del func["params"]
            fixed_funcs.append(func)
            continue

        if func.get("type") == "object" and func["properties"].get("params"):
            func["parameters"] = func["properties"]["params"]
            name_prop = func["properties"]["name"]
            if name_prop["type"] == "string":
                func["name"] = name_prop["description"]
            else:
                import pdb; pdb.set_trace()
            del func["type"]
            del func["properties"]
            continue

        import pdb; pdb.set_trace()

dataset.map(fix_functions)
# len(dataset.filter(fix_functions)["train"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

> /tmp/ipykernel_301443/3793523765.py(11)fix_functions()
      9 
     10     fixed_funcs = []
---> 11     for func in funcs:
     12         if func.get("type") == "string":
     13             continue



ipdb>  func


{'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'Helps locate information by transforming the input query into relevant search terms and filters.'}, 'parameters': {'type': 'object', 'properties': {'alternatives': {'type': 'array', 'items': {'type': 'string'}, 'description': 'A list of alternate important keywords or phrases from the input, like acronyms, common alternate synonyms, etc.'}, 'search_terms': {'type': 'array', 'items': {'type': 'string'}, 'description': 'A list of crucial keywords or phrases from the input.'}}}, 'date_range': {'type': 'object', 'properties': {'begin': {'type': 'string', 'description': 'Restricts results to items dated on or after this value, if specified in the input query.'}, 'end': {'type': 'string', 'description': 'Restricts results to items dated on or before this value, if specified in the input query.'}}}}}


ipdb>  exit
